In [1]:
%load_ext autoreload
%autoreload 2

In [10]:
import sys
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder, scale, PolynomialFeatures
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error, r2_score

# Подготовка путей

In [3]:
src_path = Path('.').absolute().parents[0]
sys.path.insert(0, f'{src_path}')
sys.path[0]

'/Users/sitd1/0_python_projects/danyad/src'

# Чтение файла

In [4]:
data_folder = src_path / 'data'

In [5]:
df = pd.read_csv(data_folder / 'data.csv', sep=';')

In [6]:
print(df.shape)
df.head(2)

(84, 18)


,Z1,Z2,X1,X2,X3,X4,X5,X6,X7,Z3,Z4,Z5,Z6,X8,Z7,Z8,X9,Y
0,Ж,Гастрошунтирование,52.021918,4.619178,45.724737,40.771224,30.483158,3500,25.529645,3,Да,Да,Да,9.136986,Нет,Комбинированная терапия без инсулина,0.075,0.059
1,Ж,Гастрошунтирование,52.046575,4.871233,45.785640,44.953174,29.136316,2100,27.887617,2,Да,Да,Да,2.238356,Нет,Монотерапия без инсулина,0.065,0.059


# Предподготовка переменных

In [7]:
target_name = 'Y'
features = df.columns.drop(target_name)
features_categorical = [el for el in features if 'z' in el.lower()]
features_numerical = features.difference(features_categorical)

# Подготовка к обучению

In [11]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), features_numerical),
        ('cat', OneHotEncoder(), features_categorical)
    ])

pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('regressor', LinearRegression())])

# Define the parameter grid for GridSearchCV
param_grid = {
    'regressor__fit_intercept': [True, False],
    'regressor__normalize': [True, False]
}

In [12]:

# Perform GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid, cv=5)
grid_search.fit(X, y)

# Print the best parameters and best score
print("Best parameters found: ", grid_search.best_params_)
print("Best score found: ", grid_search.best_score_)

NameError: name 'X' is not defined